In [ ]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import matplotlib.pyplot as plt

# Cargar el dataset de noticias falsas
file_path = "noticias_falsas_cleaned.csv"
noticias = pd.read_csv(file_path)

# Configuración del modelo de resumen
model_name = "mrm8488/bert2bert_shared-spanish-finetuned-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print(f"Total de noticias cargadas: {len(noticias)}")

In [2]:


def resumir_noticia(texto, max_length=130, min_length=30):
    """
    Función para resumir el contenido de una noticia utilizando un modelo de NLP.
    """
    texto_recortado = texto[:500]  # Limitar el texto inicial a 500 caracteres
    inputs = tokenizer.encode("summarize: " + texto_recortado, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(
        inputs, max_length=max_length, min_length=min_length, 
        length_penalty=2.0, num_beams=4, early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Dividir las noticias en bloques de 100 y resumir
for i in range(0, len(noticias), 100):
    subset = noticias.iloc[i:i+100].copy()
    subset['Resumen'] = subset['Contenido'].apply(lambda x: resumir_noticia(str(x)))

    output_file = f"falsas/noticias_falsas_resumidas_{i//100 + 1}.csv"
    subset.to_csv(output_file, index=False)
    print(f"Noticias resumidas guardadas en {output_file}")

# Combinar todos los resúmenes en un único DataFrame
path = "falsas"
csv_files = glob.glob(path + "/*.csv")

# Leer y concatenar los archivos CSV generados
df_list = (pd.read_csv(file) for file in csv_files)
big_df = pd.concat(df_list, ignore_index=True)

# Limpiar el DataFrame combinado
big_df.drop(columns=['URL'], inplace=True, errors='ignore')
big_df['word_count'] = big_df['Resumen'].apply(lambda x: len(str(x).split()))

# Filtrar resúmenes por longitud de palabras
big_df = big_df[(big_df['word_count'] > 10) & (big_df['word_count'] < 45)]

# Guardar el DataFrame final en un archivo CSV
final_output_file = "falsas_res_full.csv"
big_df.to_csv(final_output_file, index=False)
print(f"Archivo final guardado en {final_output_file}")

# Visualizar el histograma del recuento de palabras
plt.figure(figsize=(10, 6))
plt.hist(big_df['word_count'], bins=40, edgecolor='black')
plt.xlabel('Número de palabras en el resumen')
plt.ylabel('Frecuencia')
plt.title('Histograma del recuento de palabras en los resúmenes')
plt.tight_layout()
plt.show()
